## Collect data

In [ ]:
!kaggle competitions download -c aptos2019-blindness-detection

In [ ]:
!mv *.csv *.zip data/

In [ ]:
!unzip data/train_images.zip -d data/train
!unzip data/test_images.zip -d data/test

In [ ]:
!rm data/*.zip

## Process data

In [ ]:
import pandas as pd
from pathlib import Path
from fastai.vision import *
%matplotlib inline

# Data locations
path = Path("data/")

In [ ]:
test = ImageList.from_csv(path, folder="test", csv_name="test.csv", suffix=".png")
train = (ImageList.from_csv(path, folder="train", csv_name="train.csv", suffix=".png")
    .split_by_rand_pct(0.2)
    .label_from_df()
    .add_test(test)
    .transform(get_transforms(flip_vert=False), size=128)
    .databunch(path=".", bs=64)
    .normalize(imagenet_stats)
        )

In [ ]:
train.show_batch(rows=3, figsize=(7, 7))

In [ ]:
from fastai.metrics import KappaScore
kappa = KappaScore()
kappa.weights = "quadratic"

In [ ]:
learn = cnn_learner(train, models.resnet50, metrics=kappa)

In [ ]:
learn.fit_one_cycle(4)